### Symbolic Computation of Robot Dynamics by using recursive Newton-Euler Formulation


In [1]:
import sympy as sp



In [2]:
# Define symbolic variables
l1, l2 = sp.symbols('l1 l2', real=True)
q1, q2 = sp.symbols('q1 q2', real=True)
q1_dot, q2_dot = sp.symbols('q1_D q2_D', real=True)
q1_ddot, q2_ddot = sp.symbols('q1_DD q2_DD', real=True)
g = sp.Symbol('g', real=True)

# Define DH parameters
a = [0, l1]
alpha = [0, 0]
theta = [q1, q2]
d = [0, 0]

In [3]:
# Transformation Matrices
T01 = sp.Matrix([
    [sp.cos(theta[0]), -sp.sin(theta[0]), 0, a[0]],
    [sp.sin(theta[0]) * sp.cos(alpha[0]), sp.cos(theta[0]) * sp.cos(alpha[0]), -sp.sin(alpha[0]), -d[0] * sp.sin(alpha[0])],
    [sp.sin(theta[0]) * sp.sin(alpha[0]), sp.cos(theta[0]) * sp.sin(alpha[0]), sp.cos(alpha[0]), d[0] * sp.cos(alpha[0])],
    [0, 0, 0, 1]
])
T12 = sp.Matrix([
    [sp.cos(theta[1]), -sp.sin(theta[1]), 0, a[1]],
    [sp.sin(theta[1]) * sp.cos(alpha[1]), sp.cos(theta[1]) * sp.cos(alpha[1]), -sp.sin(alpha[1]), -d[1] * sp.sin(alpha[1])],
    [sp.sin(theta[1]) * sp.sin(alpha[1]), sp.cos(theta[1]) * sp.sin(alpha[1]), sp.cos(alpha[1]), d[1] * sp.cos(alpha[1])],
    [0, 0, 0, 1]
])

# Rotation Matrices
R01 = T01[:3, :3]
R12 = T12[:3, :3]

# Translation Matrices
L01 = T01[:3, 3]
L12 = T12[:3, 3]

In [4]:
# Initializing Velocities
v0 = sp.Matrix([0, 0, 0])
w0 = sp.Matrix([0, 0, 0])

# Velocities
v1 = R01.T * v0 + R01.T * w0.cross(R01.T * L01)
w1 = R01.T * w0 + sp.Matrix([0, 0, q1_dot])

v2 = R12.T * v1 + R12.T * w1.cross(R12.T * L12)
w2 = R12.T * w1 + sp.Matrix([0, 0, q2_dot])

z1 = sp.Matrix([0, 0, 1])
z2 = sp.Matrix([0, 0, 1])

In [5]:
# Initializing Accelerations
v0_dot = sp.Matrix([0, g, 0])
w0_dot = sp.Matrix([0, 0, 0])

# Accelerations
v1_dot = R01.T * (v0_dot + w0_dot.cross(L01) + w0.cross(w0.cross(L01)))
w1_dot = R01.T * w0_dot + z1 * q1_ddot + (R01.T * w0).cross(z1 * q1_dot)

v2_dot = R12.T * (v1_dot + w1_dot.cross(L12) + w1.cross(w1.cross(L12)))
w2_dot = R12.T * w1_dot + z2 * q2_ddot + (R12.T * w1).cross(z2 * q2_dot)


In [6]:
# Define mass and inertia parameters
M_1, M_2 = sp.symbols('M_1 M_2', real=True)
MX_1, MX_2 = sp.symbols('MX_1 MX_2', real=True)
XX_1, YY_1, ZZ_1 = sp.symbols('XX_1 YY_1 ZZ_1', real=True)
XX_2, YY_2, ZZ_2 = sp.symbols('XX_2 YY_2 ZZ_2', real=True)

In [7]:
# For 2nd joint
MS_2 = sp.Matrix([MX_2, 0, 0])
I_O_2 = sp.Matrix([[XX_2, 0, 0], [0, YY_2, 0], [0, 0, ZZ_2]])
forces_2 = M_2 * v2_dot + w2_dot.cross(MS_2) + w2.cross(w2.cross(MS_2))
moments_2 = I_O_2 * w2_dot + MS_2.cross(v2_dot) + w2.cross(I_O_2 * w2)
Tau_2 = sp.simplify(moments_2.T * sp.Matrix([0, 0, 1]))


In [8]:
# For 1st joint
MS_1 = sp.Matrix([MX_1, 0, 0])
I_O_1 = sp.Matrix([[XX_1, 0, 0], [0, YY_1, 0], [0, 0, ZZ_1]])
forces_1 = M_1 * v1_dot + w1_dot.cross(MS_1) + w1.cross(w1.cross(MS_1)) + R12 * forces_2
moments_1 = I_O_1 * w1_dot + MS_1.cross(v1_dot) + w1.cross(I_O_1 * w1) + (R12 * moments_2 + sp.Matrix([l1, 0, 0]).cross(R12 * forces_2))
Tau_1 = sp.simplify(moments_1.T * sp.Matrix([0, 0, 1]))

In [9]:
# Display the torques
print('Torque at Joint 1:')
Tau_1

Torque at Joint 1:


Matrix([[MX_1*g*cos(q1) + MX_2*g*cos(q1 + q2) - 2*MX_2*l1*q1_D*q2_D*sin(q2) + 2*MX_2*l1*q1_DD*cos(q2) - MX_2*l1*q2_D**2*sin(q2) + MX_2*l1*q2_DD*cos(q2) + M_2*g*l1*cos(q1) + M_2*l1**2*q1_DD + ZZ_1*q1_DD + ZZ_2*q1_DD + ZZ_2*q2_DD]])

In [10]:
print('Torque at Joint 2:')
Tau_2

Torque at Joint 2:


Matrix([[MX_2*(g*cos(q1 + q2) + l1*q1_D**2*sin(q2) + l1*q1_DD*cos(q2)) + ZZ_2*(q1_DD + q2_DD)]])